## 12.2 Wstrzykiwanie do metod i do pól (Field Injection, Method Injection)

Wykorzystamy poprzedni przykład.

### Field Injection

Tym razem wstrzykniemy obiekt `Computer` bezpośrednio do pola. Z metody `onCreate` klasy `MainActivity` pozbędziemy się `component`

In [ ]:
// val component = DaggerComputerComponent.create()
// computer = component.getComputer()

I dodajmy adnotację `@Inject` do pola `computer`

In [ ]:
@Inject
lateinit var computer: Computer

Zwróćmy uwagę na brak modyfikatora `private` - gdy chcemy wykonać wstrzyknięcie bezpośrednio do pola, musi być ono **publiczne**.

W interfejsie `ComputerComponent` usuwamy metodę `getComputer`, zamiast niej dodajemy metodę `inject`, przyjmującą jeden argument typu `MainActivity`

In [ ]:
@Component
interface ComputerComponent {
    fun inject(activity: MainActivity)
}

Przechodzimy do głównej aktywności. Jak poprzednio tworzymy `component`, tym razem wywołujemy metodę `inject` podając jako argument `this`

In [ ]:
val component = DaggerComputerComponent.create()
component.inject(this)

Przez wywołanie metody `inject` przekazujemy `MainActivity` do `Dagger2`. Automatycznie nastąpi wstrzyknięcie **wszystkich** pól oznaczonych adnotacją `@Inject`.

Minusem jest konieczność upublicznienia wszystkich pól. Zaletą jest możliwość wykonania wstrzyknięcia do wielu pól i uniknięcia pisania dla każdego osobnej metody.

### Method Injection

Ostatnim sposobem wykonywania wstrzyknięć są wstrzyknięcia przez metodę - zdecydowanie najrzadziej wykorzystywane.

Dodajmy klasę `Monitor` z jedną metodą `setComputer`

In [ ]:
class Monitor @Inject constructor() {
    fun setListener(computer: Computer): String{
        return "monitor connected"
    }
}

Dodajmy pole publiczne (pomocnicze) oraz metodę `monitor` do klasy `Computer`

In [ ]:
class Computer @Inject constructor (
    private val case: Case,
    private val gpu: GPU,
    private val cpu: CPU,
    private val motherboard: Motherboard,
    private val powerSupply: PowerSupply
        ) {

    var text: String = " "

    fun work(): String{
        return "working"
    }

    @Inject
    fun monitor(monitor: Monitor){
        text = monitor.setComputer(this)
    }
}

W głównej aktywności możemy dodać tekst do pola `TextView`

In [ ]:
textView.text = computer.work() + computer.text

### Podsumowanie

- najpopularniejsze jest wstrzykiwanie przez konstruktor
- jeżeli w klasie mamy wszystkie trzy typy wstrzyknięć, kolejność wykonania jest następująca
    - konstruktor
    - pole
    - metoda
- rzadko wykorzystuje się więcej niż jeden sposób
- wykorzystanie wszystkich trzech jest niespotykane
- jednym z niewielu zastosowań wstrzykiwania przez metodę jest sytuacja w której musimy przekazać instancję jako argument

```kotlin
        monitor.setComputer(this)
```

- jeżeli to wywołanie zostałby przeniesione do konstruktora lub blokku `init`, w efekcie dostalibyśmy `NuyllPointerException` - ponieważ staramy się przekazać obiekt który nie został jeszcze utworzony do metody ninnej klasy
- jeżeli chcemy wykonać wstrrzyknięcie do pola w klasie z dostępnym konstruktorem, nie jest konieczne wykonywanie innych operacji. Przykładowo, jeżeli chcemy wstrzyknąć `GPU` do klasy `Computer` przez pole, jedyną zmianą którą musimy wykonać jest dodanie adnotacji `@Inject` do odpowiedniego pola (pole to musi być publiczne, niefinalne)

```kotlin
        @Inject lateinit var gpu: GPU
```
- jeżeli nie mamy dostępnego konstruktora w klasie (aktywność, fragment), tworzymy obiekt o typie interfejsu za pomocą automatycznie wygenerowanej klasy

```kotlin
         val component = DaggerComputerComponent.create()
         component.inject(this)
```
- w poprzednich dwóch punktach różnicą jest metoda inicjacji - jeżeli jest dostępny konstruktor i wykonujemy również wstrzyknięcie przez konstruktor, inicjuje to cały proces. Jeżeli konstruktora nie ma, musimy zainicjować proces manualnie wywołując metodę `inject`